In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab5-mp3.ipynb")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from IPython.display import Audio
%matplotlib inline

# EE 120: Audio Signal Processing⁠⁠— MP3 Encoding
**Signals and Systems** at UC Berkeley

Acknowledgements:

- **Spring 2023** (v1.0): Naomi Sagan, Yousef Helal
- **Spring 2022** (v1.0): Naomi Sagan, Anmol Parande

# Background
Audio signals are very commonly used 1D digital signals. They cover everything from music to the dialogue from your favorite TV show. However, in order to adequately represent audio as a digital signal, we need to take many samples of the continuous time soundwave producing the audio. This is a consequence of the Nyquist criterion. The human ear can hear frequencies up to 20,000 Hz, so in order to adequately capture this frequency range, when recording digital audio, we must take at least 40,000 samples per second. The typical sampling frequency used for audio is 44.1 kHz, which is slightly above this limit.

To put this in perspective, suppose you download a 3 minute song. This is 180 seconds of audio sampled at 44.1kHz. This is close to a whopping 8 million samples! If we use a 16-bit integer to store each sample, the song in its raw form would take about 16 MB. When we store it in the raw form, we save it to a `.wav` file. However, these files will quickly become impractical to record and store because they are so huge. For this reason, we typically want to _compress_ our audio signals before saving them to a file.

The basic idea behind compression is that we can transform a signal into a _sparse_ representation. Sparse respresentations have very few nonzero values, and so we can store only the nonzero values and their locations instead of the entire signal. 

![Abstract Compression Diagram](images/compression-diagram.png)

All compression algorithms follow the same general procedure:
1. **Transform** the input signal into a sparse representation.
2. **Quantize** the result so we can represent each value of the signal with a finite number of bits.
3. **Code** the quantized coefficients into a file by assigning bits to them in an intelligent manner.

There are many compression formats for Audio, but in this lab, we will cover a basic version of the most common one: MP3. In particular, we will focus on steps 1 and 2, Transform and Quantize, because those deal most directly with EE 120. If you are interested in the "Code" piece of compression, you can take classes like EE 121, EECS 126, and EE 229.

To start, lets load a 22 second audio clip of the song "Dancing Queen" by ABBA. Note that this song was sampled at 48 kHz, not 44.1kHz like typical audio waves. A 48 kHz sampling rate is typically used when we want to capture a song in very high fidelity, which of course, we must do for ABBA to truly appreciate their musical genius.

### **Warning:** This might be loud, so turn down your volume when playing the audio.

In [ ]:
fs, song = wavfile.read('dancing_queen_clip.wav')

print("Samples: %d, Sampling Frequency: %d" % (song.size, fs))
Audio(song, rate=fs)

# The MP3 Standard
MP3, or MPEG-1 Layer III encoding, is a powerful method to compress an audio signal with very little decrease in quality.
It makes use of **psychoacoustics**, or the science of how humans perceive sound, to discard much (sometimes over 90%!) of the information in an audio file without much of an audible difference.

The full encoding scheme (from reference [1], pictured below) is complicated and we won't go over the whole thing in detail.

We will focus on the following blocks, each labeled with their role in the Compression Pipeline:
- Analysis Polyphase Filterbank, FFT, and MDCT (Transform).
- Scalar and non-uniform quantizer (Quantize)

![Full MP3 encoding scheme](images/encoding_diagram.png)

*Note: In the above diagram, "PCM input" refers to the raw audio signal.*

# Q1: MP3 Encoding

_Encoding_ refers to the process of taking a signal and going to its compressed representation. For our purposes, this means transforming it into a sparse representation and then quantizing it.

## Q1a: Background—Downsampling
Before we go over the MP3 encoding process, let's go through a key concept: downsampling. When we downsample a signal, $x(n)$, by factor $M$, we create a new signal, $w(n)$, that only contains every $M$<sup>th</sup> sample of $x$:
$$w(n) = x(Mn).$$

We will leave the frequency domain analysis as an exercise (if you want to work it out, try a process similar to the sampling of a continuous-time signal, or take EE 123) and provide the result: if $X(\omega)$ is the DTFT of $x(n)$, then we can perform the following steps to get $W(\omega)$:
1. Replicate $X(\omega)$: create $M$ shifted copies of $X(\omega)$, each centered at integer multiples of $\frac{2\pi}{M}$, and add them together. We will call the result $X_p(\omega)$.
2. Stretch $X_p(\omega)$ by a factor of $M$.

**Note**: remember that DT frequencies are $2\pi$-periodic, so all spectra here are also $2\pi$-periodic.

For a signal that is bandlimited such that it does not have frequencies higher than $\pm\frac{\pi}{M}$, downsampling would look like this (credit: old EE 123 slides):
![downsampling](images/downsample.png)

If the signal has frequencies higher than $\pm\frac{\pi}{M}$, then aliasing will occur in a manner similar to sampling of a CT signal (but don't worry about that, we'll make sure that no aliasing occurs in this lab!).

### Your Job
Fill in the function `downsample`, which performs downsampling in the time domain. It should only take one [list slice](https://www.geeksforgeeks.org/python-list-slicing/) operation!

In [ ]:
def downsample(x, M):
    """
    Downsamples signal x by factor M.
    
    Inputs:
    - x: discrete-time signal
    - M: factor at which to downsample.
    
    Output:
    - w: M-downsampled signal w(n) = x(Mn).
    As a sanity check, if x(n) has length N, w(n) will have length of roughly M/N.
    """
    w = ...
    return w

In [ ]:
grader.check("q1a")

## Q1b: Analysis Uniform Modulated Filterbank
The first step of MP3 encoding is the _Analysis Polyphase Filterbank_. Polyphase Filterbanks are covered in detail in EE 123, but the basic idea is that this block uses a set of filters (a filter bank) to split the signals into its sub-bands. Each sub-band is an equal-sized section of the spectrum of the original signal which only overlaps with other sub-bands at the boundaries.

Below is an example of a spectrum divided into four sub-bands.
![sub-bands](images/subbands.png)

If the original time-domain signal is length $N$, then each sub-band will also be length $N$. If there are $M$ sub-bands, then we need to store $NM$ total samples. MP3 is supposed to be a compression algorithm, so we don't want to increase the amount of data we store!

It turns out we can perform careful downsampling of each sub-band to only store $\frac{N}{M}$ samples per sub-band while ensuring that we don't lose any information about the original signal (see **Step 2** and **Step 3** for more details).

<!-- Suppose the original signal is of length $N$. In the time-domain, each sub-band will also be of length $N$, but in the Fourier Domain, each sub-band only occupies $\frac{2\pi}{M}$ of the entire spectrum, where $M$ is the number of sub-bands. Since each sub-band only occupies $\frac{2\pi}{M}$ of the entire spectrum, if we downsample each sub-band in the time-domain, it will then occupy the full frequency range $[-\pi, \pi]$. This means that we can split our signal into sub-bands and still keep the same number of total samples in memory. I.e after we downsample each sub-band, we will have $M$ sub-bands, each of length $\frac{N}{M}$, leading to $N$ total samples.
Note that downsampling does not cause any aliasing due to the fact that each sub-band is non-overlapping and equally-sized, so we can still reconstruct our original signal perfectly. -->

A polyphase filterbank is a specific set of filters which accomplishes the goal in a very efficient manner. However, for our purposes, we can implement an equivalent though less efficient filterbank called a **uniform modulated filterbank** (reference [3]).

### Step 1: Division into Subbands
The first step of the Uniform Modulated Filterbank is to divide the audio signal, $x(n)$ into $M=32$ subbands. What this means is we want to find signals $x_k(n)$ such that $$\sum_{k=0}^{31} x_k(n) = x(n),$$
where each sub-band represents equal-sized sections of the spectrum of the original signal.
Specifically, the $k$<sup>th</sup> sub-band will include frequencies from $-(k+1)\frac{\pi}{M}$ through $-k\frac{\pi}{M}$ and $k\frac{\pi}{M}$ through $(k+1)\frac{\pi}{M}$.

Since the sub-bands overlap at the boundary frequencies ($\pm (k+1) \frac{\pi}{M}$ and $\pm k\frac{\pi}{M}$), we will scale these boundaries by $\frac{1}{2}$ so that they add together to get the original signal.

For ease of implementation, we will split each sub-band into two segments: one with positive frequencies and one with negative frequencies:

$$X_k(\omega) = X_{k,\text{upper}}(\omega) + X_{k,\text{lower}}(\omega),$$
where
$$X_{k,\text{upper}} = \begin{cases} 
    X(\omega), & \text{ if } \frac{\pi k}{M} < \omega < \frac{\pi (k+1)}{M} \\
    \frac{1}{2}X(\omega), & \text{ if } \omega = \frac{\pi k}{M}, \frac{\pi (k+1)}{M} \\
    0, \text{ else.}
\end{cases},\quad X_{k,\text{lower}} = \begin{cases} 
    X(\omega), & \text{ if } -\frac{\pi (k+1)}{M} < \omega < -\frac{\pi k}{M} \\
    \frac{1}{2}X(\omega), & \text{ if } \omega = -\frac{\pi k}{M}, -\frac{\pi (k+1)}{M} \\
    0, \text{ else.}
\end{cases}.$$

**Important note**: We can use conjugate symmetry to show that $X_{k,\text{upper}}(\omega)$  because $X_{k,\text{lower}}(\omega) = X_{k,\text{upper}}^*(-\omega)$. So, we only have to calculate the upper half of each sub-band!

<!-- We need to design filters $H_k$ which takes $x$ as an input and produce $x_k$. Consider the bandpass filter -->

<!-- $$H_k(\omega) = \begin{cases}
    \frac{1}{2}, & \text{if }\omega = \frac{\pi}{M}k\text{ or }\frac{\pi}{M}(k + 1) \\
    1,& \text{if }\frac{\pi}{M}k < \omega < \frac{\pi}{M}(k + 1) \\
    0,& \text{else.}
\end{cases}$$

$H_k(\omega)$ will isolate the positive part of the $k-$th subband, producing 
$Y_k(\omega) = H_k(\omega)X(\omega)$. If we apply $H_k(-\omega)$, then we get the negative part of the subband $Z_k(\omega) = H_k(-\omega)X(\omega)$. 

Since $x(n)$ is real, $X(-\omega) = X^*(\omega)$, so $Z_k(-\omega) = Y^*(\omega)$. Therefore, the kth subband is

$$X_k(\omega) = Y_k(\omega) + Z_k(\omega) = Y_k(\omega) + Y_k^*(-\omega) \leftrightarrow x_k(n) = y_k(n) + y_k^*(n) = 2\mathfrak{Re}\{y_k(n)\}$$

Therefore, we only need to care about finding $y_k(n)$ in order to find the $k-$th sub-band. -->

### Step 2: Modulation
Now that we have 32 sub-bands, all that is left is to downsample them. Otherwise we will be left with a $32-$fold increase in samples being stored, which is the opposite of compression!

However, as mentioned in **1(a)**, horrific aliasing will occur unless unless the highest frequency present in each $X_k(\omega)$ is $\pm\frac{\pi}{M}$.

Unfortunately, $X_k$ is bandlimited by $\frac{\pi(k+1)}{M}$. For instance, for some signal divided into 4 sub-bands, $X_2$ could look like:
![X3](images/X3.png)

However, if we modulate each part of $X_k$ such that the positive frequencies fill the space $\left[0, \frac{\pi}{M}\right]$ and the negative frequencies fill the space $\left[-\frac{\pi}{M}, 0\right]$, then we can downsample without aliasing.
After this modulation, $X_2$ will be as follows:
![X3](images/X3_mod.png)

<!-- There is an edge case, however; *some aliasing does occur*: when we modulate, the frequencies $\frac{\pi}{M}k$ and $-\frac{\pi}{M}k$ add at $0$. And, after we downsample, the frequencies $\frac{\pi}{M}(k+1)$ and $-\frac{\pi}{M}(k+1)$  add at $\pi$. If $W_k(\omega)$ is the spectrum of the downsampled signal, we have 

$$W_k(0) = X_k\left(\frac{\pi}{M}k\right) + X_k\left(-\frac{\pi}{M}k\right) = 2 \mathfrak{Re}\left\{X_k\left(\frac{\pi}{M}k\right)\right\} \text{ and } W_k(\pi) = 2 \mathfrak{Re}\left\{X_k\left(\frac{\pi}{M}(k+1)\right)\right\}.$$

This means $X\left(\frac{\pi}{M}k\right), k \in \{1, \dots, M-1\}$ could have an imaginary component that is lost when we modulate and downsample. -->
There is a slight edge case here: $X_k\left(\frac{\pi}{M}k\right)$ and $X_k\left(-\frac{\pi}{M}k\right)$  add together at $\omega = 0$.
You can apply conjugate symmetry to see that $X_k\left(\frac{\pi}{M}k\right) + X_k\left(-\frac{\pi}{M}k\right) = 2 \mathfrak{Re}\left\{X_k\left(k\frac{\pi}{M}\right)\right\}$.
So, the imaginary component of $X_k\left(\pm\frac{\pi}{M}k\right)$ is lost when we modulate.

In real MP3 encoding, special filters mitigate this issue.
But for our purposes, we will store the imaginary component of each $X_k\left(\frac{\pi}{M}k\right)$ in an array. (**You don't have to worry about this part—we'll handle it for you!**)

### Step 3: Downsampling
Now, we can use the `downsample` function from 1(a) to downsample each (modulated) sub-band.

### Your Job
1. Fill in the function `split_subbands`. This function will implement the procedure described above as follows:

```
Take the FFT of x(n) to get X_DFT
for k = 0, ..., M-1:
    1. Initialize Xk_upper_modulated to be all zero.
    2. Set the beginning indices of Xk_upper_modulated to the section 
       of X_DFT corresponding to frequencies in the interval [(k)pi/M, (k+1)pi/M].
    3. Divide Xk_upper_modulated by two at the edges of the sub-band.
    4. Recover the time-domain sub-band using Xk_upper_modulated. We did this step for you.
```

2. Fill in the line `downsampled_subbands[i] = ...` in `uniform_mod_filterbank`, to produce an $M$-downsampled version of each sub-band. You should call the `downsample` function you wrote in 1(a).

<!-- ### Small example for illustration:
Suppose you had 2 sub-bands and a length 8 signal, and the DFT of $x(t)$ was as follows:

$$\begin{bmatrix}
a & b & c & d & e & d^* & c^* & b^*
\end{bmatrix}$$
The indices of the DFT correspond to the following frequencies:
- Index 0 corresponds to $\omega = 0$
- Index 1: $\omega = \frac{2\pi}{N} = \frac{\pi}{4}$
- Index 2: $\omega = 2\frac{2\pi}{N}= \frac{\pi}{2}$
- Index 3: $\omega = 3\frac{2\pi}{N} = 3\frac{\pi}{4}$.
- Index 4: $\omega = 4\frac{2\pi}{N} = \pi$. DT frequencies are $2\pi$ periodic, so this corresponds to $\omega = -\pi$ as well.
- Index 5: $\omega = 5\frac{2\pi}{N} = (N - 3)\frac{2\pi}{N}$. DT frequencies are $2\pi$ periodic, so this is equivalent to $-3\frac{2\pi}{N} = -3\frac{\pi}{4}$.
- Index 6: $\omega = (N-2)\frac{2\pi}{N}$ or $-2\frac{2\pi}{N} = -\frac{\pi}{2}$.
- Index 7: $\omega = (N-1)\frac{2\pi}{N}$ or $-\frac{2\pi}{N} = -\frac{\pi}{4}$.

The positive half of the first sub-band ($X_{0,\text{upper}}$) includes frequencies $0$ through $\frac{\pi}{2}$:

$$\begin{bmatrix}
\frac{c}{2} & d & \frac{e}{2} & 0 & 0 & 0 & 0 & 0
\end{bmatrix}$$

after modulation.

- We recommend focusing on the positive half of the spectrum,

$$\begin{bmatrix}
a & b & c & d & e & 0 & 0 & 0
\end{bmatrix},$$

which would look like 

$$Y_k = \begin{bmatrix}
\frac{c}{2} & d& \frac{e}{2} & 0 & 0 & 0 & 0 & 0
\end{bmatrix}$$

after filtering and modulation.
To recover the full sub-band, you can compute 

$$x_k(n) = 2 \mathfrak{Re}\{y_k(n)\}$$

in the time domain, as described in the **Division into Subbands** section.
- We highly recommend that you do as much of `split_subbands` in the frequency domain as possible. -->

In [ ]:
def split_subbands(x, M):
    """
    Splits a signal into M equally-spaced sub-bands
    in the frequency domain.
    Inputs:
    - x: raw audio signal
    - M: number of subbands
    Returns: a tuple consisting of
    - A length-M array, where each element is time-domain representation
      of the corresponding subband.
    - The imaginary component of the spectrum of x, evaluated
      at omega = pi/M * k.
    """
    # For simplicity, zero-pad x such that x.size is a multiple of 2M
    x = np.concatenate((x, np.zeros(-x.size % (2 * M))))
    N = x.size # Length of the time-domain signal
    
    # Number of DFT coefficients in the positive-frequency half of the sub-band
    # The negative-frequency half will have the same number of DFT coefficients
    subband_width = N // (2 * M) 
    
    subbands = [];
        
    # Take the FFT of x
    X_DFT = ...
    
    # Iterate over each sub-band
    for k in range(M):
        Xk_upper_modulated = np.zeros(N, dtype='complex128')
        
        # Find index corresponding to frequency kpi/M
        start_index_of_subband = ...
        # Find index corresponding to frequency (k+1)pi/M
        end_index_of_subband = ...
        
        # Get the section of X_DFT corresponding to the range [kpi/M, (k+1)pi/M]
        ...
        
        # Divide the edges of the sub-band by 2
        Xk_upper_modulated[[0, subband_width]] /= 2
        
        # Apply conjugate symmetry to get the full sub-band by just using Xk_upper
        # Theory for why this works is left as an exercise.
        time_domain_subband = 2 * np.real(np.fft.ifft(Xk_upper_modulated))
        
        # Add the sub-band to the array
        subbands.append(time_domain_subband)
        
    subbands = np.array(subbands)
    imag_components = np.imag(np.fft.fft(x))[:N//2+1:N // (2 * M)]
    return subbands, imag_components

In [ ]:
def uniform_mod_filterbank(x, M):
    """
    A uniform modulated filterbank.
    Splits x into M sub-bands and then downsamples by M.
    Inputs:
    - x: raw audio signal
    - M: number of subbands
    Returns:
    - A tuple with the following elements:
        - A length-M array, where each element is the downsampled
          subband in the time domain.
        - The imaginary component of the spectrum of x, evaluated
          at omega = pi/M * k.
    """
    subbands, imag_components = split_subbands(x, M)
    downsampled_subbands = []
    for i in range(subbands.shape[0]):
        # Call your downsample function!
        downsampled_subband_i = ...
        downsampled_subbands.append(downsampled_subband_i)
    
    return np.array(downsampled_subbands), imag_components

In [ ]:
grader.check("q1b")

For the rest of the lab, we will be working with the subbands of our song. Depending on how fast your machine (or Datahub) is, this cell may take some time to run. For reference, it only takes a few seconds on a 2017 Macbook Pro.

In [ ]:
subbands, imag_components = uniform_mod_filterbank(song, 32)

## Q1c: Windowing
After dividing the signal into its sub-bands, we now want to divide each sub-band into chunks and take the MDCT of each chunk. The MDCT is another technique to transform a signal into the frequency domain (more on this later). We don't take the MDCT of the entire sub-band because that wouldn't give us a good picture of how the frequency content of the signal changes over time, and this would lead to low compressability and bad reconstructions of the original signal. However, we need to be careful about how we split each sub-band into chunks.

Consider a signal $x(n)$. If you split it into equally-sized chunks $x_k(n)$, that is identical to multipling it with a set of signals $v_k(n)$ where
$$ v_k(n) = \begin{cases} 1, & \text{ if n is in chunk k } \\ 0 & \text{ else.} \end{cases} $$

In the frequency domain, for each resulting chunk $x_k(n) = v_k(n)x(n)$, this corresponds to $X_k(\omega) = \frac{1}{2\pi} V_k * X(\omega)$, where $V_k(\omega)$ is a sinc function. This sinc function will cause what we call "spectral leakage" because the side-lobes of the sinc will smear the low frequency components into the high frequencies, causing high frequency distortions when we perform the convolution. 

Below is the magnitude of $V_k(\omega)$, in decibels ($20\log_{10}|V_k(\omega)|$). Note the prominent side-lobes!
![sinc lobes](images/sinc_lobes.png)

To avoid this, we use special window functions which minimize the amount of spectral leakage that occurs.

For example, say we use a rectangular windowing function to isolate the blue section of the following sine wave.
<img src="images/sin_to_window.png" width=700px/>

The periodic extension of the window would be as follows:

<img src="images/blocked_sin.png"/>

Note the sharp discontinuity at the end of each period! In the Fourier domain, this is from the added high-frequency components from the aliasing.

Now, let us multiply that same block with a better window function $W(n)$
$$W(n) = \sin\left(\frac{\pi}{N}\left(n + \frac{1}{2}\right)\right).$$
There are many window functions that may be used (reference [2]), but this one was specificlly chosen to work well with the next step, the modified discrete cosine transform (MDCT, reference [4]).

The periodic extension is (for the most part) smooth, corresponding to reduced high-frequency spectral leakage.

<img src="images/smooth_windowed_sin.png"/>

In the MP3 standard, the size of the windows we take depend on a "psychoacoustic model" which uses the psychology of human hearing to determine the optimal window length. Fast changes in the spectrum of the signal would lead to shorter windows. To simplify things, we will implement a constant **window size of 512 samples**.


### Overlapping Windows

When we window each subband of the audio signal, we will do so in a special way.
The MDCT, which will be described in the next section, requires adjacent windows to overlap by a factor of of $\frac{1}{2}$, *i.e.* for the second half of one window to overlap with the first half of the next. So when we window each sub-band, our windows should overlap by 256 samples.

### Your Job
1. Fill in `window_fn`, which takes in window size $N$ and returns $W(n) = \sin\left(\frac{\pi}{N}\left(n + \frac{1}{2}\right)\right)$ as a length-$N$ numpy array.
2. Fill in two lines of `apply_window`, which takes in a 1D signal `x` and splits it into windows of size `window_size`, overlapping according to `overlap_ratio`.
The function will return an array where each element is an array of length `window_size` containing a section of `x` multiplied with the output of `window_fn`.

In [ ]:
def window_fn(N):
    """
    Returns a length-N window function, as defined in this question's preamble.
    Input: length of the window
    Output: length-N numpy array representing the window
    """    
    ### YOUR CODE HERE
    n = ...
    w = ...
    return w

In [ ]:
def apply_window(x, window_fn, window_size, overlap_ratio):
    """
    Applys windowing to a signal x.
    Inputs:
    - x: signal to be windowed
    - window_fn: window function to apply. Takes in the window_size
                 and returns the window as a numpy array.
    - window_size: length of the Hann window
    - overlap_ratio: overlap between adjacent windows, represented
                     as a ratio of the window size
    Return: a list where each element is a length-window_size window
            of the signal x.
    """
    windows = []
    
    # If the signal cannot be perfectly covered by the windows,
    # zero-pad the end so that it can.
    window_sep = int((1-overlap_ratio)*window_size)
    if (x.size - window_size) % window_sep:
        last_window_start =  (x.size // window_sep) * window_sep
        x = np.concatenate((x, np.zeros(window_size + last_window_start - x.size)))
    ## End zero-padding
    
    ### Iterate over the indexes where each window begins 
    for window_start in np.arange(0, x.size - window_size + 1, (1 - overlap_ratio) * window_size):
        start_idx = int(window_start)
        
        # Shift x to the left such that the start of the window is at the beginning of the array,
        # and then slice the result such that it is of length window_size
        x_shifted_cropped = ...
        
        # Multiply x_shifted_cropped by the window function of the appropriate size
        window = ...
        
        windows.append(window)
        
    return windows

In [ ]:
grader.check("q1c")

## Q1d: MDCT (Modified Discrete Cosine Transform)
Next, we move each window into the frequency domain using the Modified Discrete Cosine Transform.
The MDCT is a specialized variation of the DFT that uses cosines as basis functions instead of complex exponentials. Because cosines are the basis function, the MDCT of a real signal always has real coefficients. This is important for compression because if we had imaginary coefficients, we would have to store both the real and imaginary piece, essentially doubling the amount of information we need to store.


Unlike some other variations on the DCT (discrete cosine transform), the MDCT is **lapped**, *i.e.*, designed to work well on blocks of data where consecutive blocks overlap by a factor of $\frac{1}{2}$.

See the section on the IMDCT (inverse MDCT) and time-domain aliasing cancellation for an in-depth description of where lapping comes into play.
Most notably, the lapping in the MDCT allows us to perfectly reconstruct our signal even with a non-rectangular windowing function applied: In smoothing the edges of a window, the window function we use distorts our signal. However, for windowing functions that meet certain conditions (more on this later), we can window, take the MDCT, take the IMDCT, and then get our original signal back.

The analysis equation for the MDCT is as follows:
$$X_k, k \in \{0, \dots, N-1\} = \sum_{n=0}^{2N-1} x(n) \cos\left[\frac{\pi}{N} \left(n + \frac{1}{2} + \frac{N}{2}\right)\left(k + \frac{1}{2}\right)\right].$$

Note that the MDCT takes in $2N$ real inputs and produces $N$ real coefficients.
This is due to the principle underlying the MDCT, **time-domain aliasing cancellation (TDAC)**.
We discuss TDAC in some detail in the appendix, but it's pretty complicated. You can also look at reference [4] for more general MDCT information.

**Your Job**: Fill in `MDCT`, which applys the MDCT to `x`, which is a length-$2N$ block of a real signal (`x` will, in the case of MP3 encoding, be a window of a subband).

In [ ]:
def MDCT(x):
    """
    Performs the MDCT on signal x.
    Inputs:
    - x: real-valued input signal of length 2N.
    Return: a length-N numpy array containing the MDCT coefficients of x.
    """
    # For simplicity, make the length of x even.
    if x.size % 2 == 1:
        x = np.concatenate((x, np.zeros(1)))
    Xk = np.zeros(x.size//2)
    
    ...
    return Xk

In [ ]:
grader.check("q1d")

Below, we apply windowing and the MDCT to each subband. For reasons we will once again defer to the Time-Domain Aliasing Cancellation section, we need to zero-pad the beginning and end of $x(n)$ with $N$ zeros on each side to perfectly reconstruct the whole signal with the IMDCT.

Let's apply our windowed MDCT to the subbands we created earlier. This cell might take up to several minutes to run depending on your machine.

In [ ]:
def apply_windowed_MDCT(x, window_fn, N):
    # Zero-pad the beginning and end for perfect reconstruction
    # See the IMDCT and Time-Domain Aliasing Cancellation section
    # for an explanation of why.
    x = np.concatenate((np.zeros(N), x, np.zeros(N)))
    return [MDCT(window) for window in apply_window(x, window_fn, 2*N, 1/2)]

MDCT_windowed_subbands = np.array([apply_windowed_MDCT(band, window_fn, 256) for band in subbands])

# Q1e: Quantization

After all of this work, we have finally transformed our audio signal into a sparse representation. Exactly how sparse is it though? Let's see what percent of values are non-zero.

In [ ]:
np.count_nonzero(MDCT_windowed_subbands) / MDCT_windowed_subbands.size

Over 99% of our values are non-zero! This representation of the signal is not sparse at all! Our algorithm must be broken, right?

Before we throw our computer at the wall, lets take a close look at the magnitudes of our coefficients by plotting the average value of the magnitudes of the coefficients in each sub-band on a logarithmic scale.

In [ ]:
plt.figure(figsize=(15, 7))

plt.semilogy(np.mean(np.abs(MDCT_windowed_subbands), axis=(1, 2)))
plt.ylabel("Average MDCT Coefficient Value")
plt.xlabel("Sub-band")
plt.show()

At sub-band 20 and beyond, the MDCT coefficients are incredibly small! Moreover, the coefficients in sub-bands 15 - 20 are small relative to the coefficients in sub-bands 1 - 14. Sub-band 15 and above represent frequencies higher than ~11kHz, and sub-band 20 and above represent frequencies higher than 15 kHz. Human speech has frequencies up to 8 kHz, and beyond 15 kHz, human hearing degrades rapdily, so it makes sense that these sub-bands have smaller coefficient values. Afterall, why would an artist put effort into making music at these high frequencies if their audience can't even hear them. How can we use this information to build better compression?

The higher the frequency of a sound, the less sensitive humans are to changes in amplitude of that frequency. What that implies is we can achieve better compression rates by **quantizing** the coefficients in each sub-band. As an example, suppose we force every coefficient to be a multiple of 10. Then coefficients between 0 and 9.999999 will be quantized to 0, coefficients between 10 and 19.9999999 will be quantized to 1, etc. If we have a lot of coefficients in the range 0 to 9.999999, then that brings us more compressive power because these coefficients will all become 0.

In the cell below, we have given you a function called quantize. It takes in a quantization table, which is an array that has 1 value per sub-band, and the Windowed MDCT coefficients of a signal. It then quantizes the signal by floor dividing the coefficients in each sub-band by the corresonding value in the quantization table.

In [ ]:
def quantize(quant_table, coeffs):
    quant_table = quant_table[:, np.newaxis, np.newaxis]
    return np.sign(coeffs) * (np.abs(coeffs) // quant_table)

Let's try quantizing our signal using two different quantization tables.

**Your Job**
1. Quantize `MDCT_windowed_subbands` according to a uniform quantization table. That is, every entry in the quantization is equal to 1.
2. Quantize `MDCT_windowed_subbands` according to a linear quantization table. That is, the ith entry of the quantization table is equal to i, where $1 \leq i \leq 32$

**Hint**: We have 32 subbands, so your quantization tables should have 32 entries in them.

In [ ]:
uniform_quantized_coeffs = ...
linear_quantized_coeffs = ...

In [ ]:
grader.check("q1e")

Let's look at what percent of our coefficients are non-zero after quantization under each scheme.

In [ ]:
uniform_quantization_nonzero_count = np.count_nonzero(uniform_quantized_coeffs) / MDCT_windowed_subbands.size
linear_quantization_nonzero_count = np.count_nonzero(linear_quantized_coeffs) / MDCT_windowed_subbands.size

print(f"Percentage of non-zero coeffients after uniform quantization: {np.round(uniform_quantization_nonzero_count, 5) * 100}")
print(f"Percentage of non-zero coeffients after linear quantization: {np.round(linear_quantization_nonzero_count, 5) * 100}")

<!-- BEGIN QUESTION -->

**Q: Assuming that you can reconstruct an equally good quality recording from each of these quantization schemes, which one would you use? Why?**

**A:** (TODO)

<!-- END QUESTION -->

# Q2: MP3 Decoding
So far, we have successfully represented our audio signal using a relatively small number of non-zero coefficients. But compression would be pretty useless if we couldn't take our compressed representation and reconstruct the original signal. This procedure is called _decoding_.

In order to decode the signal into something we can listen to, we need to reverse the effects of 
1. The Quantization
2. The MDCT
3. The Uniform Modulated Analysis Filter Bank

In the quantization step, we floor-divided our coefficients by a quantization table. To re-quantize the coefficients and put them on the same scale as they were before, we can simply multiply by the quantization table! We have given you the `requantize` function to use for this purpose.

Note that while requantization puts our coefficients on the same scale as they were before, we still lost information that is irrecoverable. For example, suppose a coefficent had the value 1.5. After quantization by 1, it would have value 1. Requantizing it would leave the value at 1 because we have no way of knowing it was 1.5 before quantization. This is why MP3 is known as _lossy_ compression. In lossy compression, we throw away information, so we can never get an exact reconstruction, but we can get a reconstruction that is "good enough".

In [ ]:
def requantize(quantized_coeffs, quant_table):
    quant_table = quant_table[:, np.newaxis, np.newaxis]
    return quantized_coeffs * quant_table

After requantizing, the next step is to undo the MDCT by using the IMDCT. The IMDCT is given by the synthesis equation
$$x(n), n \in \{0, \dots, 2N-1\} = \frac{1}{N} \sum_{k=0}^{N-1} X_k \cos\left[\frac{\pi}{N} \left(n + \frac{1}{2} + \frac{N}{2}\right)\left(k + \frac{1}{2}\right)\right]$$

If you are interested in learning more about MDCT and IMDCT, please take a look at the **Appendix**!

<br><br>
**Your job**:
* Implement the IMDCT() function below, according to the equation given above


In [ ]:
def IMDCT(Xk):
    """
    Performs the IMDCT on MDCT coefficients Xk.
    Inputs:
    - Xk: N real-valued MDCT coefficients.
    Return: a length-2N numpy array corresponding to IMDCT{Xk}.
    """
    x = np.zeros(Xk.size*2)
    
    ### YOUR CODE HERE
    ...
    return x

def windowed_IMDCT(Xk, window_fn):
    N = Xk.size
    return 2 * window_fn(2*N) * IMDCT(Xk)

In [ ]:
def reconstruct(IMDCT_outputs, length=None):
    """
    Given the outputs of the IMDCT of overlapping blocks of a signal
    (denoted x_hat in the equations above), reconstruct the original
    signal x(n).
    Inputs:
    - IMDCT_outputs: list of outputs of the IMDCT, where each index
                     corresponds to a block of the signal.
    - length: the length of the final signal, used if zeros were 
              added onto the original signal to accommodate windowing. 
              If not None, the reconstructed signal will be sliced such 
              that it has "length" elements.
    Return: reconstructed x(n)
    """
    N = IMDCT_outputs[0].size // 2
    n_windows = len(IMDCT_outputs)
    x = np.zeros((n_windows-1)*N)
    
    for i in range(n_windows-1):
        x[N*i:N*(i+1)] = IMDCT_outputs[i][N:] + IMDCT_outputs[i+1][:N]
    return x if length is None else x[:length]

In [ ]:
grader.check("q2a")

### Synthesis Filter Bank
Finally, we will reverse the operations done by the analysis filter bank. We have given you the code that does this, and it is present below. If you would like to learn more about how this code works, please check out the "Synthesis Filter Bank" section of the appendix.

In [ ]:
def synthesis_filter_bank(subbands, imag_components):
    # Take the FFT of each subband
    subband_X = [np.fft.fft(band) for band in subbands]
    M = len(subbands)
    N = subbands[0].size * M
    x = np.zeros(N, dtype='complex128')
    
    subband_width = N//(2*M)
    for k in range(M):
        # This is the Zk from step A of the procedure above
        Zk = np.zeros(N, dtype='complex128')
        
        # Xk stores the coefficients of the kth sub-band
        Xk = subband_X[k]
        
        Xk[[0, subband_width]] /= 2
        Zk[subband_width*k:subband_width*(k+1)+1] = M * Xk[:subband_width+1]
        Zk[-subband_width*k] += M * Xk[0]
        Zk[N-subband_width*(k+1):N-subband_width*k] += M * Xk[subband_width:]
            
        # Add in the imaginary component stored by the analysis filter bank.
        Zk[subband_width*k] += 1j*imag_components[k]
        # Apply conjugate symmetry
        Zk[-subband_width*k] -= 1j*imag_components[k]
        x += np.fft.ifft(Zk)
        
    return np.real(x)

## Q2b: Putting it All Together

Now that we can reconstruct our signal, lets look at the reconstructions of the song based on the two different quantization schemes.

In [ ]:
def decode_coeffs(coeffs, imag_at_pi, quant_table):
    coeffs = requantize(coeffs, quant_table)
    IMDCT_blocks = [[windowed_IMDCT(block, window_fn) for block in band] for band in coeffs]
    reconstructed = [reconstruct(band, song.size//32) for band in IMDCT_blocks]
    song_hat = synthesis_filter_bank(reconstructed, imag_at_pi)
    return song_hat

_Type your answer here, replacing this text._

<!-- BEGIN QUESTION -->



In [ ]:
uniform_quant_table = ...
linear_quant_table = ...

uniform_quantized_song = decode_coeffs(uniform_quantized_coeffs, imag_components, uniform_quant_table)
linear_quantized_song = decode_coeffs(linear_quantized_coeffs, imag_components, linear_quant_table)

In [ ]:
Audio(uniform_quantized_song, rate=fs)

In [ ]:
Audio(linear_quantized_song, rate=fs)

**Q: Earlier, we made a decision about which quantization scheme we wanted to use without listening to the audido quality of the reconstruction. Now that you can hear the audio quality of the reconstruction, does your answer change which quantization scheme you want to use?**

**A**: (TODO)

<!-- END QUESTION -->

# Conclusions

This lab gives you a taste of the type of signal proccessing work which goes into compression. Some of the heavier theory is covered in more detail in EE 123. One important factor to note is that throughout the lab, we have been assuming that more zero coefficients means greater compression. For the most part, this is accurate, but your actual level of compression heavily depends on how you do the "Coding" piece of the compression pipeline. Nevertheless, more zeros is always a good thing. Moreover, we neglected to consider the imaginary parts of the sub-band boundaries in our analysis of our compression algorithm. In practice, these would also need to be stored, but given the length of the audio file, an additional 32 numbers is not a great deal of overhead and does not impact our compression capabilities in practice. 

The MP3 standard goes far beyond what we have covered in this lab, which is why it can achieve astounding compression rates. If you're curious, checkout some of the references below. You can also try loading in your own wav files and trying to see how our solution compares to MP3.

## References
[1] Raissi, Rassol. “The Theory Behind MP3.” mp3-Tech.org, http://www.mp3-tech.org/programmer/docs/mp3_theory.pdf.

[2] Understanding Ffts and Windowing - Ni. https://download.ni.com/evaluation/pxi/Understanding%20FFTs%20and%20Windowing.pdf.

[3] Chapter 2: Time-Frequency Mapping Tools in MP3 and MPEG 2/4. https://ir.nctu.edu.tw/bitstream/11536/50035/6/756506.pdf. 

[4] “Modified Discrete Cosine Transform.” Wikipedia, Wikimedia Foundation, 17 Sept. 2021, https://en.wikipedia.org/wiki/Modified_discrete_cosine_transform. 

# Appendix

### **The MDCT and DCT-IV**

The MDCT is a modification of the DCT-IV, whose synthesis and analysis equations are
$$\hat{X}_k = \sum_{n=0}^{N-1} x(n) \cos\left[\frac{\pi}{N} \left(n + \frac{1}{2}\right)\left(k + \frac{1}{2}\right)\right].$$
$$x(n) = \frac{1}{N} \sum_{k=0}^{N-1} \hat{X}_k \cos\left[\frac{\pi}{N} \left(n + \frac{1}{2}\right)\left(k + \frac{1}{2}\right)\right].$$
It can be shown that the DCT-IV basis functions have even symmetry about $n = -\frac{1}{2}$ and odd symmetry about $n = N - \frac{1}{2}$.

We would like to relate the MDCT to the DCT-IV, as it is known that the DCT-IV synthesis and analysis equations are perfect inverses of each other, meaning $\text{IDCT}\{\text{DCT}\{x(n)\}\} = x(n)$.

Let us denote the DCT-IV basis function as
$$\chi_k \in \mathbb{R}^n = \cos\left[\frac{\pi}{N} \left(n + \frac{1}{2}\right)\left(k + \frac{1}{2}\right)\right].$$
Utilizing the symmetry properties of the DCT-IV, it can be shown that the MDCT basis functions $\psi_k \in \mathbb{R}^{2N}$ relate to the DCT-IV basis functions as follows:
$$\psi_k\left[n=0:\frac{N}{2}\right] = \chi_k\left[\frac{N}{2}:N\right] \\
\psi_k\left[\frac{N}{2}:\frac{3N}{2}\right] = \chi_k\left[N:2N\right] = -\text{reverse}\left\{\chi_k\right\} \\
\psi_k\left[\frac{3N}{2}:2N\right] = \chi_k\left[2N:\frac{5N}{2}\right] = -\chi_k\left[0:\frac{N}{2}\right]
$$

Define the input signal $x(n) \in \mathbb{R}^{2N}$ as $[a, b, c, d]$, such that $x\left[0:\frac{N}{2}\right] = a$, $x\left[\frac{N}{2}:N\right] = b$, *etc.*
If we apply the MDCT, $a$ is projected onto $\chi_k\left[\frac{N}{2}:N\right]$, $[b, c]$ is projected onto $-\text{reverse}\left\{\chi_k\right\}$, and $d$ is projected onto $-\chi_k\left[0:\frac{N}{2}\right]$.
So, $\text{MDCT}\{x(n)\} = \text{DCT}\{[-d - \text{reverse}\{c\}, a - \text{reverse}\{b\}]\}$.

**Time-Domain Aliasing Cancellation**

The IMDCT is the $\frac{1}{2}$ times a version of the DCT-IV synthesis equation that is extended to length $2N$ and shifted to the left by $\frac{N}{2}$. 
It can be shown through similar logic as in the previous paragraph that
$$\text{IMDCT}\{\text{MDCT}\{x(n)\}\} = \frac{1}{2}[a - \text{reverse}\{b\}, b - \text{reverse}\{a\}, c + \text{reverse}\{d\}, d + \text{reverse}\{c\}].$$

Although the IMDCT does not directly reconstruct $x(n)$, we can do so via the principle of time-domain aliasing cancelling.
This uses the fact that consecutive blocks of $x(n)$ are overlapped by a factor of $\frac{1}{2}$.
Consider the signal
$$x(n) = [a, b, c, d, e, f],$$
where each block is of length $\frac{N}{2}$.
Taking the MDCT and inverse MDCT on the overlapped length-$2N$ sections $[a, b, c, d]$ and $[c, d, e, f]$, we have
$$\text{IMDCT}\{\text{MDCT}\{[a, b, c, d]\}\} = \frac{1}{2}[a - \text{reverse}\{b\}, b - \text{reverse}\{a\}, c + \text{reverse}\{d\}, d + \text{reverse}\{c\}]\\
\text{IMDCT}\{\text{MDCT}\{[c, d, e, f]\}\} = \frac{1}{2}[c - \text{reverse}\{d\}, d - \text{reverse}\{c\}, e + \text{reverse}\{f\}, f + \text{reverse}\{e\}].$$
If we add the second half of $\text{IMDCT}\{\text{MDCT}\{[a, b, c, d]\}\}$ to the first half of $\text{IMDCT}\{\text{MDCT}\{[c, d, e, f]\}\}$, we get $[c, d]$. This is a perfect reconstruction of the center section of $x(n)$!

$[a, b]$ and $[e, f]$ cannot be fully reconstructed because they do not ovelap with anything. **This is why we zero-padded the ends of the signal in `apply_windowed_MDCT`!**
The $N$ elements at the beginning and end of the signal that we cannot reconstruct are just the zeros that we added, so we can reconstruct the full song with the IMDCT.

### **Implementing IMDCT and TDAC**
**IMDCT for Windowed Signals**

We'll forego the derivation for this part, but, for blocks of a signal produced by a window function $W(n) \in \mathbb{R}^{2N}$ such that $W[n=0:N]^2 + \text{reverse}\{W[N:2N]\}^2 = \begin{bmatrix} 1 & \cdots & 1\end{bmatrix}$, the IMDCT is
$$x(n) = W(n)\frac{2}{N} \sum_{k=0}^{N-1} X_k \cos\left[\frac{\pi}{N} \left(n + \frac{1}{2} + \frac{N}{2}\right)\left(k + \frac{1}{2}\right)\right],$$
and $x(n)$ can be reconstructed using the same time-domain aliasing cancellation technique as with non-windowed signals.

**We can apply TDAC to reconstruct overlapped sections of the original signal as follows**:

Assume we have a function $x(n)$ split into length-$2N$ blocks, where the second half of one block overlaps with the first half of the next, and that we have taken the MDCT of each block. Denote the $i$<sup>th</sup> block as $x_i(n)$. Also assume that we padded the beginning and end with $N$ zeros each before windowing.
- Take the IMDCT of each block. The $i$<sup>th</sup> IMDCT output will be called $\hat{x}_i(n) \in \mathbb{R}^{2N}$.
- $x(n)$ can be reconstructed as follows:
    $$x[n=iN:(i+1)N] = \hat{x}_{i+1}[0:N] + \hat{x}_{i}[N:2N].$$
    If there are $K$ windows, we perform this process for $i=0$ to $K-1$.

<br>

### **Synthesis Filter Bank**
Finally, we will reverse the operations done by the analysis filter bank. We have given you the code that does this, and it is present below
Recall that the the $k$<sup>th</sup> subband was produced by appying the filter 

$$H_k(\omega) = \begin{cases}
    \frac{1}{2},& \omega = \frac{\pi}{M}k\text{ or }\frac{\pi}{M}(k + 1) \\
    1,& \frac{\pi}{M}k < \omega < \frac{\pi}{M}(k + 1) \\
    0,& \text{else}
\end{cases}$$

to the signal, shifting the spectrum of the result to the left by $\frac{\pi}{M}k$, adding the conjugate of the result in the time-domain, and downsampling by $M$.

Below, we have enumerated the process of implementing the synthesis filter bank.
As before, we will only focus on positive frequencies, *i.e.*, $\omega \in [0, \pi]$ or the first half of the DFT coefficients.
1. **Upsample each subband by $M$.**

    In the time domain, if $y_k(n)$ is a $M$-upsampled version of $x_k(n)$, we have
    $$y_k(n) = \begin{cases}
        0,&n \text{ mod } M \neq 0 \\
        x_k\left(\frac{n}{M}\right),&n \text{ mod } M = 0
    \end{cases}$$
    In the frequency domain, we have 
    $$Y_k(\omega) = X_k(M\omega).$$
    In terms of the DFT coefficients,$ Y_{k, i} = X_{k, i\text{ mod } \frac{N}{M}}$ where $Y_{k, i}$ is the ith DFT coefficient of $y_k(n)$ and $X_{k, i}$ is the ith DFT coefficient of $x_k(n)$.
2. **Interpolate each frequency band.**

    As in time-domain interpolation, apply a low-pass filter with cutoff frequencies $\pm \frac{\pi}{M}$ and scaling factor $M$.
    The result is that the DFT coefficients are now
    $$Y_{k, i} = \begin{cases}
        M X_{k, i}, &-\frac{N}{2M} \leq i \leq \frac{N}{2M} \\
        0, &\text{otherwise}
    \end{cases}$$

3. **Shift the spectrum.**

    To reverse the frequency-shifting done in the analysis filter bank, shift the right half of the spectrum to the right by $\frac{\pi}{M}k$ and the left half of the spectrum to the left by $\frac{\pi}{M}k$.
    
    When doing this, we have to consider the following issues:
    - The frequency $0$ in $Y_k(\omega)$ will be mapped to both $\frac{\pi}{M}k$ and $-\frac{\pi}{M}k$.
    - For $k=0$, $\frac{\pi}{M}k$ and $-\frac{\pi}{M}k$ will be the same frequency ($\omega = 0$).
    - The highest frequency in one sub-band overlaps with the lowest frequency in the subesquent sub-band.
    - For the last sub-band, $\frac{\pi}{M} (k+1)$ and $-\frac{\pi}{M} (k+1)$ are the same frequency ($\omega = \pi$).



## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit.

Please upload the zip file produced by the result of this command to Gradescope.

In [ ]:
grader.export(force_save=True, run_tests=True)